In [53]:
import ee
import geemap
import ee.mapclient
import folium

project_id = 'твой айди'

# Аутентификация и инициализация Earth Engine
ee.Authenticate()
ee.Initialize(project=project_id)


aoi = ee.Geometry.Point([63.0952911376953, 61.9183006286621])

# 2. Параметры визуализации
viz = {'min': 0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}

# 3. Определите даты
startDate1 = ee.Date.fromYMD(2023, 6, 1)
endDate1 = ee.Date.fromYMD(2023, 9, 30)

# Sentinel-2
# 4. Загрузите и отфильтруйте коллекцию Sentinel-2
S2_1 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(startDate1, endDate1) \
    .filterBounds(aoi) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5) \
    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8'])

# 5. Создайте медианное изображение, разделите на 10000 и обрежьте
img1 = ee.Image(S2_1.median()).divide(10000).set('period', '1');


# 6. Визуализация с помощью folium
# Получите карту ID и токен для визуализации
map_id_dict = img1.getMapId(viz)

# Создайте карту folium
m = folium.Map(location=[61.9183006286621, 63.0952911376953], zoom_start=12)  # Замените lat, lon на центр вашей области
# долгота, широта, масштаб

# Добавьте слой с изображением на карту folium
folium.raster_layers.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    name='True Color 1'
).add_to(m)

# Добавьте контроль слоев, чтобы можно было включать/выключать слои
m.add_child(folium.LayerControl())

# Отобразите карту
display(m)



In [52]:
import ee
import geemap
import ee.mapclient
import folium

project_id = 'твой айди'

# Аутентификация и инициализация Earth Engine
ee.Authenticate()
ee.Initialize(project=project_id)


aoi = ee.Geometry.Point([63.0054016113281, 61.940990447998])

# 2. Параметры визуализации
viz = {'min': 0, 'max': 0.3, 'bands': ['SR_B4', 'SR_B3', 'SR_B2']}

# 3. Определите даты
startDate1 = ee.Date.fromYMD(2023, 6, 1)
endDate1 = ee.Date.fromYMD(2023, 9, 30)

# Landsat
L_9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterDate(startDate1, endDate1) \
    .filterBounds(aoi) \
    .select(['SR_B2', 'SR_B3', 'SR_B4'])
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  return image.addBands(optical_bands, None, True)
L_9 = L_9.map(apply_scale_factors)

# 5. Создайте медианное изображение, разделите на 10000 и обрежьте
imgL = ee.Image(L_9.median())


# 6. Визуализация с помощью folium
# Получите карту ID и токен для визуализации
map_id_dict = imgL.getMapId(viz)

# Создайте карту folium
m = folium.Map(location=[61.940990447998, 63.0054016113281,], zoom_start=12)  # Замените lat, lon на центр вашей области
# долгота, широта, масштаб

# Добавьте слой с изображением на карту folium
folium.raster_layers.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    name='True Color 1'
).add_to(m)

# Добавьте контроль слоев, чтобы можно было включать/выключать слои
m.add_child(folium.LayerControl())

# Отобразите карту
display(m)

In [68]:

# Аутентификация в Google Earth Engine
import ee
import ee.mapclient
import geemap
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

project_id = 'твой айди'

# Аутентификация и инициализация Earth Engine
ee.Authenticate()
ee.Initialize(project=project_id)

# 1. Определение области интереса (AOI) и даты
aoi = ee.Geometry.Point([63.0429801940917, 61.9103202819824])

start_date = '2023-05-01'
end_date = '2023-07-31'

# 2. Загрузка и подготовка обучающих данных (полигоны свалок из GeoPackage)

# Путь к GeoPackage (загрузите его в Colab или Google Drive)
gpkg_path = '/content/svalki.gpkg'  # Замените на фактический путь
layer_name = 'svalki'       # Замените на имя слоя с полигонами свалок в GeoPackage

gdf = gpd.read_file(gpkg_path, layer=layer_name)
print(gdf.head()) # Проверьте, что данные загрузились правильно

# Убедитесь, что у вас есть столбец с метками классов (например, 'class', 'landfill', 'is_landfill')
# Если нет, создайте его на основе других атрибутов, если необходимо
# Пример:  Предположим, что у вас есть столбец 'type', где 'landfill' означает свалку
# gdf['class'] = gdf['type'].apply(lambda x: 1 if x == 'landfill' else 0)

# Проверка наличия столбца 'class'
if 'param' not in gdf.columns:
    print("Error: Column 'class' not found in GeoPackage.  Please add a 'class' column (1 for landfill, 0 for not landfill).")
    exit()

# b) Преобразование GeoDataFrame в FeatureCollection Earth Engine

def geojson_to_ee_feature(feature):
    """Converts a GeoJSON Feature to an Earth Engine Feature and casts 'class' to int."""
    geom = feature['geometry']
    props = feature['properties']
    # Преобразуем 'class' в целое число
    try: # если class это int или float
        props['param'] = int(props['param'])
    except ValueError: # если class это строка
        try:
            props['param'] = int(float(props['param']))
        except ValueError:
            print(f"can't convert param label {props['param']} to integer")
            raise # кидаем оригинальную ошибку

    return ee.Feature(ee.Geometry(geom), props)

# Преобразование GeoDataFrame в GeoJSON
geojson = gdf.__geo_interface__

# Создание списка ee.Feature из GeoJSON
ee_features = [geojson_to_ee_feature(f) for f in geojson['features']]

# Создание FeatureCollection из списка ee.Feature
features = ee.FeatureCollection(ee_features)

# ВАЖНО: Проверка FeatureCollection - убедитесь, что столбец 'class' существует
first_feature = features.first()
print("First feature properties:", first_feature.getInfo()['properties'])

# 3. Выбор снимков ДЗЗ (Sentinel-2 или Landsat)
# Пример использования Sentinel-2:
image_collection = (ee.ImageCollection('COPERNICUS/S2_SR')
                    .filterDate(start_date, end_date)
                    .filterBounds(aoi)
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))  # Фильтр по облачности

# Выбор нужных каналов (bands) - определите после фильтрации!
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12'] # Sentinel-2 bands: Blue, Green, Red, NIR, SWIR1, SWIR2

# Обработка изображений с разными наборами каналов (при необходимости)
def select_bands(image):
    return image.select(bands)

image_collection = image_collection.map(select_bands)

# Выбор медианного изображения (или другого подходящего способа композиции)
composite = image_collection.median()  # Теперь composite должен иметь только нужные bands
print(f"composite bands: {composite.bandNames().getInfo()}")  # Печать, чтобы убедиться в наборе каналов

# 4. Подготовка признаков (features) для машинного обучения
# ВАЖНО: Укажите правильное имя столбца с классами!
correct_class_name = 'param'  # <----- ЗАМЕНИТЕ НА ПРАВИЛЬНОЕ ИМЯ СТОЛБЦА

training = composite.sampleRegions(
    collection=features,
    properties=[correct_class_name],  # <----- ИСПОЛЬЗУЙТЕ ПРАВИЛЬНОЕ ИМЯ СТОЛБЦА
    scale=10  # Разрешение пикселя (Sentinel-2: 10m, Landsat: 30m)
)

# 5. Разделение данных на обучающую и тестовую выборки
training_data = training.randomColumn()
train_set = training_data.filter(ee.Filter.lt('random', 0.7))
test_set = training_data.filter(ee.Filter.gte('random', 0.7))

# **ОТЛАДКА: ПРОВЕРКА РАСПРЕДЕЛЕНИЯ КЛАССОВ**
print("Train set class distribution:", train_set.aggregate_histogram(correct_class_name).getInfo()) #  <----- ИСПОЛЬЗУЙТЕ ПРАВИЛЬНОЕ ИМЯ СТОЛБЦА
print("Test set class distribution:", test_set.aggregate_histogram(correct_class_name).getInfo()) #  <----- ИСПОЛЬЗУЙТЕ ПРАВИЛЬНОЕ ИМЯ СТОЛБЦА

# 6. Обучение модели машинного обучения (Random Forest)
classifier = ee.Classifier.smileRandomForest(10)  # 10 деревьев
classifier = classifier.train(
    features=train_set,
    classProperty=correct_class_name,  #  <----- ИСПОЛЬЗУЙТЕ ПРАВИЛЬНОЕ ИМЯ СТОЛБЦА
    inputProperties=bands
)

# 7. Оценка точности модели
test_accuracy = test_set.classify(classifier).errorMatrix(correct_class_name, correct_class_name) #  <----- ИСПОЛЬЗУЙТЕ ПРАВИЛЬНОЕ ИМЯ СТОЛБЦА
print('Матрица ошибок теста: ', test_accuracy.getInfo())
print('Точность теста: ', test_accuracy.accuracy().getInfo())

# 8. Классификация изображения
classified_image = composite.classify(classifier)
classified_image = classified_image.toInt()  # <--- Добавлено: Явное преобразование в int

# Добавлено: детальная статистика classified_image
stats = classified_image.reduceRegion(
    reducer=ee.Reducer.autoHistogram(),  # Используем autoHistogram для более подробной информации
    geometry=aoi,
    scale=10,
    maxPixels=1e9
)

print("Detailed Classified Image Stats:", stats.getInfo())


# 9. Визуализация результатов
# Создание карты с помощью geemap
Map = geemap.Map()
Map.centerObject(aoi, 12)
Map.addLayer(composite, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2 Composite')
# Добавлено: min и max для визуализации
Map.addLayer(classified_image, {'min': 0, 'max': 1, 'palette': ['green', 'red']}, 'Landfill Classification')
Map.addLayer(features, {'color': 'yellow'}, 'Training Data') # Отображаем обучающие данные
Map  # Отображение карты в Colab

   param                                           geometry
0      0  MULTIPOLYGON (((63.17497 61.92115, 63.17529 61...
1      0  MULTIPOLYGON (((63.17819 61.92285, 63.17951 61...
2      0  MULTIPOLYGON (((63.09271 61.91573, 63.09285 61...
3      0  MULTIPOLYGON (((63.09199 61.91539, 63.09185 61...
4      0  MULTIPOLYGON (((63.09192 61.91534, 63.09169 61...
First feature properties: {'param': 0}
composite bands: ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
Train set class distribution: {'0': 2446, '1': 462}
Test set class distribution: {'0': 1055, '1': 190}
Матрица ошибок теста:  [[1055, 0], [0, 190]]
Точность теста:  1
Detailed Classified Image Stats: {'classification': [[0, 1]]}


Map(center=[61.9103202819824, 63.0429801940917], controls=(WidgetControl(options=['position', 'transparent_bg'…